In [1]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

Executing: /tmp/apt-key-gpghome.lny1R5COKw/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.XO7DxpYHZ2/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.P46F5Zx5jS/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 http://deb.debian.org/debian bust

# Import Libraries

In [2]:
import pandas as pd
import os
import requests
import time
import random

from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Initialize Selenium driver

In [3]:
WEBDRIVER_DELAY_TIME_INT = 10

service = Service(executable_path=r'/usr/bin/chromedriver')
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.headless = True
driver = webdriver.Chrome(service=service, options=chrome_options)
driver.implicitly_wait(5)
wait = WebDriverWait(driver, WEBDRIVER_DELAY_TIME_INT)

In [4]:
datasets = []
deletion_script = 'arguments[0].parentNode.removeChild(arguments[0]);'
for page_idx in tqdm(range(1, 5)):
  main_url = f'https://www.thivien.net/searchpoem.php?PoemType=16&ViewType=1&Country=2&Age[]=3&Page={page_idx}'
  driver.get(main_url)

100%|██████████| 4/4 [00:14<00:00,  3.64s/it]


In [5]:
content_tags_xpath = '//*[@class="page-content container"]/div[2]/div/div[@class="list-item"]'
content_tags = driver.find_elements(By.XPATH, content_tags_xpath)

# Poem Content Extract

In [6]:
for idx in range(len(content_tags)):
    content_tag_xpath = f'/html/body/div[4]/div[2]/div/div[{2+idx}]'
    content_title_xpath = f'/html/body/div[4]/div[2]/div/div[{2+idx}]/h4/a'
    content_tag = wait.until(
    EC.presence_of_element_located(
    (By.XPATH, content_tag_xpath)
    )
    )

    poem_title = wait.until(
    EC.presence_of_element_located(
    (By.XPATH, content_title_xpath)
    )
    ).text

    poem_url = wait.until(
    EC.presence_of_element_located(
    (By.XPATH, content_title_xpath)
    )
    ).get_attribute('href')

    try:
        driver.get(poem_url)

        poem_src_xpath = '//div[@class="small"]'
        poem_content_tag = wait.until(
        EC.presence_of_element_located(
        (By.CLASS_NAME, 'poem-content')
        )
        )

        try:
            poem_content_i_tag = poem_content_tag.find_element(
            By.TAG_NAME,
            'i'
            )
            driver.execute_script(deletion_script, poem_content_i_tag)
        except:
            pass

        try:
            poem_content_b_tag = poem_content_tag.find_element(
            By.TAG_NAME,
            'b'
            )
            driver.execute_script(deletion_script, poem_content_b_tag)
        except:
            pass

        poem_content = poem_content_tag.text

        try:
            poem_src_tag = wait.until(
            EC.presence_of_element_located(
            (By.XPATH, poem_src_xpath)
            )
            )
            poem_src = poem_src_tag.text
        except:
            poem_src = ''
        poem_info = {
        'title': poem_title,
        'content': poem_content,
        'source': poem_src,
        'url': poem_url
        }
        datasets.append(poem_info)
        driver.back()
    except Exception as e:
        print(e)
        print(poem_url)

In [7]:
df = pd.DataFrame(datasets)
df.to_csv('poem_dataset.csv', index=True)

In [8]:
df.head()

,title,content,source,url
0,“Những lúc mình vui sướng”,Những lúc mình vui sướng\nHãy thầm hỏi trong l...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
1,“Nơi chính tâm cơn bão”,Nơi chính tâm cơn bão\nDễ tìm được bình tâm,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Nguy%E1%BB%85n-V%C5%A9...
2,“Quyết tâm và nghị lực”,Quyết tâm và nghị lực\nGiúp người ta đứng lên\...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
3,“Ta buồn khi phải thấy”,Ta buồn khi phải thấy\nNhiều người bỏ rơi ta\n...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
4,“Tài là do thiên phú”,Tài là do thiên phú\nĐức là do mình rèn\nHai c...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...


# Build Model

In [9]:
!pip install -qq datasets==2.16.1 evaluate==0.4.1 transformers[sentencepiece]==4.35.2
!pip install -qq accelerate==0.26.1
!apt install git-lfs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 6.4 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 40 not upgraded.


In [10]:
import os
import math
import torch
import pandas as pd

from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import DataCollatorForLanguageModeling
from transformers import TrainingArguments, Trainer
from huggingface_hub import notebook_login
from datasets import Dataset

In [11]:
DATASET_PATH = '/content/poem_dataset.csv'
df = pd.read_csv(DATASET_PATH)
df

,Unnamed: 0,title,content,source,url
0,0,“Những lúc mình vui sướng”,Những lúc mình vui sướng\nHãy thầm hỏi trong l...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
1,1,“Nơi chính tâm cơn bão”,Nơi chính tâm cơn bão\nDễ tìm được bình tâm,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Nguy%E1%BB%85n-V%C5%A9...
2,2,“Quyết tâm và nghị lực”,Quyết tâm và nghị lực\nGiúp người ta đứng lên\...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
3,3,“Ta buồn khi phải thấy”,Ta buồn khi phải thấy\nNhiều người bỏ rơi ta\n...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
4,4,“Tài là do thiên phú”,Tài là do thiên phú\nĐức là do mình rèn\nHai c...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
5,5,“Tâm kệ”,Gội rửa sạch trần hồng\nnhư hoa ướp trầm xông\...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Ph%E1%BA%A1m-Thi%C3%AA...
6,6,“Tha lỗi lầm cho bạn”,"Tha lỗi lầm cho bạn\nLà việc tốt, tất nhiên\nN...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
7,7,“Thời gian như tiền bạc”,Thời gian như tiền bạc\nAnh sống qua một ngày\...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
8,8,“Tình yêu của bà mẹ”,Tình yêu của bà mẹ\nLặng lẽ và vô bờ\nKhông đò...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
9,9,"“Trong tình yêu, xa cách”","Trong tình yêu, xa cách\nLà phép thử nhiệm màu...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...


In [12]:
def split_content(content):
    samples = []
    poem_parts = content.split('\n\n')
    for poem_part in poem_parts:
        poem_in_lines = poem_part.split('\n')
        if len(poem_in_lines) == 4:
            samples.append(poem_in_lines)

    return samples

df['content'] = df['content'].apply(lambda x: split_content(x))
df

,Unnamed: 0,title,content,source,url
0,0,“Những lúc mình vui sướng”,"[[Những lúc mình vui sướng, Hãy thầm hỏi trong...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
1,1,“Nơi chính tâm cơn bão”,[],[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Nguy%E1%BB%85n-V%C5%A9...
2,2,“Quyết tâm và nghị lực”,"[[Quyết tâm và nghị lực, Giúp người ta đứng lê...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
3,3,“Ta buồn khi phải thấy”,"[[Ta buồn khi phải thấy, Nhiều người bỏ rơi ta...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
4,4,“Tài là do thiên phú”,"[[Tài là do thiên phú, Đức là do mình rèn, Hai...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
5,5,“Tâm kệ”,"[[Gội rửa sạch trần hồng, như hoa ướp trầm xôn...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Ph%E1%BA%A1m-Thi%C3%AA...
6,6,“Tha lỗi lầm cho bạn”,"[[Tha lỗi lầm cho bạn, Là việc tốt, tất nhiên,...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
7,7,“Thời gian như tiền bạc”,"[[Thời gian như tiền bạc, Anh sống qua một ngà...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
8,8,“Tình yêu của bà mẹ”,"[[Tình yêu của bà mẹ, Lặng lẽ và vô bờ, Không ...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
9,9,"“Trong tình yêu, xa cách”","[[Trong tình yêu, xa cách, Là phép thử nhiệm m...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...


In [13]:
df_exploded = df.explode('content')
df_exploded.reset_index(drop=True, inplace=True)
df_exploded = df_exploded.dropna(subset=['content'])
df_exploded

,Unnamed: 0,title,content,source,url
0,0,“Những lúc mình vui sướng”,"[Những lúc mình vui sướng, Hãy thầm hỏi trong ...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
2,2,“Quyết tâm và nghị lực”,"[Quyết tâm và nghị lực, Giúp người ta đứng lên...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
3,3,“Ta buồn khi phải thấy”,"[Ta buồn khi phải thấy, Nhiều người bỏ rơi ta,...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
4,4,“Tài là do thiên phú”,"[Tài là do thiên phú, Đức là do mình rèn, Hai ...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
5,4,“Tài là do thiên phú”,"[Có tài không có đức, Là tai hoạ cho đời, Khôn...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
6,5,“Tâm kệ”,"[Gội rửa sạch trần hồng, như hoa ướp trầm xông...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Ph%E1%BA%A1m-Thi%C3%AA...
7,5,“Tâm kệ”,"[Thân mặc áo giải thoát, lìa tập nhiễm phù vân...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Ph%E1%BA%A1m-Thi%C3%AA...
8,5,“Tâm kệ”,"[Thắp lửa vô lượng quang, sáng soi bảy cửa ngụ...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Ph%E1%BA%A1m-Thi%C3%AA...
9,5,“Tâm kệ”,"[Khơi lò thanh tịnh trầm, gió đưa mười cõi thơ...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Ph%E1%BA%A1m-Thi%C3%AA...
10,5,“Tâm kệ”,"[Như trái dâng cúng Phật, kết hạt chắc Bồ Đề, ...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Ph%E1%BA%A1m-Thi%C3%AA...


In [14]:
df_exploded['content'] = df_exploded['content'].apply(lambda x: '\n'.join(x))
df_exploded

,Unnamed: 0,title,content,source,url
0,0,“Những lúc mình vui sướng”,Những lúc mình vui sướng\nHãy thầm hỏi trong l...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
2,2,“Quyết tâm và nghị lực”,Quyết tâm và nghị lực\nGiúp người ta đứng lên\...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
3,3,“Ta buồn khi phải thấy”,Ta buồn khi phải thấy\nNhiều người bỏ rơi ta\n...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
4,4,“Tài là do thiên phú”,Tài là do thiên phú\nĐức là do mình rèn\nHai c...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
5,4,“Tài là do thiên phú”,Có tài không có đức\nLà tai hoạ cho đời\nKhông...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
6,5,“Tâm kệ”,Gội rửa sạch trần hồng\nnhư hoa ướp trầm xông\...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Ph%E1%BA%A1m-Thi%C3%AA...
7,5,“Tâm kệ”,Thân mặc áo giải thoát\nlìa tập nhiễm phù vân\...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Ph%E1%BA%A1m-Thi%C3%AA...
8,5,“Tâm kệ”,Thắp lửa vô lượng quang\nsáng soi bảy cửa ngục...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Ph%E1%BA%A1m-Thi%C3%AA...
9,5,“Tâm kệ”,Khơi lò thanh tịnh trầm\ngió đưa mười cõi thơm...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Ph%E1%BA%A1m-Thi%C3%AA...
10,5,“Tâm kệ”,Như trái dâng cúng Phật\nkết hạt chắc Bồ Đề\nn...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Ph%E1%BA%A1m-Thi%C3%AA...


In [15]:
TEST_SIZE = 0.1
poem_dataset = Dataset.from_pandas(df_exploded)
poem_dataset = poem_dataset.train_test_split(test_size=TEST_SIZE)

In [16]:
MODEL_NAME = 'danghuy1999/gpt2-viwiki'
tokenizer = GPT2Tokenizer.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/773k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/431k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/916 [00:00<?, ?B/s]

In [17]:
tokenizer.pad_token = tokenizer.eos_token
MAX_SEQ_LEN = 100

def preprocess_function(row):
    return tokenizer(
            row['content'],
            max_length=MAX_SEQ_LEN,
            padding='max_length',
            truncation=True
            )

tokenized_poem_dataset = poem_dataset.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=poem_dataset['train'].column_names,
)

Map (num_proc=4):   0%|          | 0/21 [00:00<?, ? examples/s]

num_proc must be <= 3. Reducing num_proc to 3 for dataset of size 3.


Map (num_proc=3):   0%|          | 0/3 [00:00<?, ? examples/s]

In [18]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Training

In [19]:
model = GPT2LMHeadModel.from_pretrained(MODEL_NAME)

pytorch_model.bin:   0%|          | 0.00/510M [00:00<?, ?B/s]

In [20]:
training_args = TrainingArguments(
  output_dir='gpt2_viet_poem_generation',
  save_strategy='epoch',
  learning_rate=2e-5,
  num_train_epochs=10,
  weight_decay=0.01,
  fp16=True
)

In [21]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_poem_dataset['train'],
    eval_dataset=tokenized_poem_dataset['test'],
    data_collator=data_collator,
    tokenizer=tokenizer
)

trainer.train()

Step,Training Loss


TrainOutput(global_step=30, training_loss=6.811069742838542, metrics={'train_runtime': 139.7325, 'train_samples_per_second': 1.503, 'train_steps_per_second': 0.215, 'total_flos': 10717056000000.0, 'train_loss': 6.811069742838542, 'epoch': 10.0})

# Inference

In [26]:
prompt = 'Học học nữa học mãi\n'
device ='cuda' if torch.cuda.is_available() else 'cpu'
inputs = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
outputs = model.generate(
    inputs,
    max_new_tokens=50,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=0.8,
    repetition_penalty=1.2
)
results = tokenizer.batch_decode(outputs, skip_special_tokens=True)
results = results[0]
print()
for line in results.split('\n'):
    print(line)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Học học nữa học mãi
Tại một người ấy, người còn thấy, người muốn muốn
và nhiều người khác càng gia lên để gặp cả hai chị
hay quan lại càng có quan chân trời
Cả hai cái, cái, cái đều là một cái
cho biết ai
